In [ ]:
pip install matplotlib

In [ ]:
pip install seaborn

In [ ]:
import psycopg2
import pandas as pd
import psycopg2.extras
import seaborn as sns
import numpy as np

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
class PostgresConnection(object):
    def __init__(self):
        self.connection = psycopg2.connect(database="ecomdb",
                                           user = "postgres",
                                           password = "cse18133",
                                           host = "localhost",
                                           port = "5432")

    def getConnection(self):
        print("Connection to DB established!")
        return self.connection

In [ ]:
con = PostgresConnection().getConnection()

# Q1: Find the division wise total_sale_price joining fact table and respective dimension table

In [ ]:
con = PostgresConnection().getConnection()
cur = con.cursor()
select_stmt = "SELECT s.division, SUM(t.total_price) " \
              "FROM star_schema.fact_table t " \
              "JOIN star_schema.store_dim s on s.store_key=t.store_key " \
                "GROUP BY CUBE(s.division) " \
                "ORDER BY s.division "
cur.execute(select_stmt)
records = cur.fetchall()

In [ ]:
records

In [ ]:
df_q1 = pd.DataFrame(records, columns=['division', 'sales'])
df_q1

In [ ]:
df_q1 =df_q1.drop(7)

In [ ]:
df_q1

In [ ]:
def changetype(x):
    x['sales']=x['sales'].astype('float64')  

In [ ]:
changetype(df_q1)

In [ ]:
df_q1

In [ ]:
df_q1=df_q1.set_index(['division'])
df_q1

In [ ]:
df_q1.plot.pie(y='sales')

In [ ]:
plt.figure(figsize=(10,5))
df_q1.plot.bar(y='sales')

In [ ]:
con = PostgresConnection().getConnection()
cur = con.cursor()
select_stmt = "SELECT s.year, SUM(t.total_price) " \
              "FROM star_schema.fact_table t " \
              "JOIN star_schema.time_dim s on s.time_key=t.time_key " \
                "GROUP BY CUBE(s.year) " \
                "ORDER BY s.year "
cur.execute(select_stmt)
records = cur.fetchall()

In [ ]:
records

In [ ]:
df_q1 = pd.DataFrame(records, columns=['year', 'sales'])


In [ ]:
df_q1

In [ ]:
df_q1 =df_q1.drop(8)
df_q1

In [ ]:
df_q1['year'] = df_q1['year'].astype('int64')
df_q1['sales'] = df_q1['sales'].astype('float64')
df_q1

In [ ]:
df_q1=df_q1.set_index(['year'])
df_q1.plot.pie(y='sales')

In [ ]:
plt.figure(figsize=(10,5))
df_q1.plot.bar(y='sales')

Q2: Find the customer/bank/transaction(cash/online) wise total_sale_price joining fact table and respective dimension table 

In [ ]:
con = PostgresConnection().getConnection()
cur = con.cursor()
select_stmt = "SELECT tr.bank_name, SUM(t.total_price)"\
                "FROM star_schema.fact_table t "\
                "JOIN star_schema.trans_dim tr on tr.payment_key=t.payment_key "\
                "GROUP BY CUBE(tr.bank_name) "\
                "ORDER BY tr.bank_name "
cur.execute(select_stmt)
records = cur.fetchall()



In [ ]:
records

In [ ]:
df_q2_bank = pd.DataFrame(records, columns=['bank_name', 'sales'])
df_q2_bank

In [ ]:
# drop the row with none value
df_q2_bank = df_q2_bank.dropna()
df_q2_bank


In [ ]:
df_q2_bank.dtypes

In [ ]:
changetype(df_q2_bank)

In [ ]:
# df_q2_bank = df_q2_bank.set_index(['bank_name'])

In [ ]:
# plt.figure(figsize=(30,15))
# df_q2_bank.plot.bar(y='sales')

In [ ]:
plt.figure(figsize=(20,12))
plt.title('Sales by Bank')
plt.xlabel('Bank Name')
plt.ylabel('Sales')
df_q2_bank.sort_values(by=['sales'], ascending=False, inplace=True)
plt.stem(df_q2_bank['sales'],df_q2_bank['bank_name'])
plt.show()


Transaction Type

In [ ]:
con = PostgresConnection().getConnection()
cur = con.cursor()
select_stmt = "SELECT tr.trans_type, SUM(t.total_price)" \
                "FROM star_schema.fact_table t " \
                "JOIN star_schema.trans_dim tr on tr.payment_key=t.payment_key " \
                "GROUP BY CUBE(tr.trans_type) " \
                "ORDER BY tr.trans_type "
cur.execute(select_stmt)
records = cur.fetchall()

In [ ]:
records

In [ ]:
df_q2_payment_type = pd.DataFrame(records,columns=['payment_type','sales'])
df_q2_payment_type

In [ ]:
df_q2_payment_type = df_q2_payment_type.dropna()
df_q2_payment_type

In [ ]:
df_q2_payment_type.dtypes

In [ ]:
changetype(df_q2_payment_type)


In [ ]:
df_q2_payment_type = df_q2_payment_type.set_index(['payment_type'])

In [ ]:
plt.figure(figsize=(30,15))
df_q2_payment_type.plot.bar(y='sales')

In [ ]:
# pie chart
df_q2_payment_type.plot.pie(y='sales')

## Q3: Total sales in Barisal

In [ ]:
con = PostgresConnection().getConnection()
cur = con.cursor()
select_stmt = "SELECT s.division, SUM(t.total_price) " \
                "FROM star_schema.fact_table t " \
                "JOIN star_schema.store_dim s on s.store_key=t.store_key " \
                "WHERE s.division = 'BARISAL' " \
                "GROUP BY CUBE(s.division) " \
                "ORDER BY s.division "
                
                
cur.execute(select_stmt)
records = cur.fetchall()



In [ ]:
records

In [ ]:
df_q3 = pd.DataFrame(records, columns=['division', 'sales'])
df_q3

In [ ]:
df_q3 =df_q3.dropna()
df_q3

## Q4: Total sales in 2015

In [ ]:
con = PostgresConnection().getConnection()
cur = con.cursor()
select_stmt = "SELECT t.year, SUM(f.total_price) " \
                "FROM star_schema.fact_table f " \
                "JOIN star_schema.time_dim t on t.time_key=f.time_key " \
                "WHERE t.year = 2015 " \
                "GROUP BY CUBE(t.year) " \
                "ORDER BY t.year "
cur.execute(select_stmt)
records = cur.fetchall()


In [ ]:
records

In [ ]:
df_q4 = pd.DataFrame(records, columns=['year', 'sales'])
df_q4

In [ ]:
df_q4 =df_q4.dropna()
df_q4

In [ ]:
df_q4['year'] = df_q4['year'].astype('int64')
df_q4['sales'] = df_q4['sales'].astype('float64')

In [ ]:
df_q4

## Q5: Total sales of Barisal in 2015

In [ ]:
con = PostgresConnection().getConnection()
cur = con.cursor()
select_stmt = "SELECT s.division, t.year,SUM(f.total_price) AS total_sales " \
    "FROM star_schema.fact_table f " \
    "JOIN star_schema.store_dim s ON f.store_key = s.store_key " \
    "JOIN star_schema.time_dim t ON f.time_key = t.time_key " \
    "WHERE s.division = 'BARISAL' AND t.year = 2015 " \
    "GROUP BY CUBE(s.division, t.year) " \
    "ORDER BY s.division, t.year "
    

cur.execute(select_stmt)
records = cur.fetchall()

In [ ]:
records

In [ ]:
df_q5 = pd.DataFrame(records, columns=['division', 'year', 'sales'])
df_q5

In [ ]:
df_q5 = df_q5.dropna()
df_q5

In [ ]:
df_q5['year'] = df_q5['year'].astype('int64')
df_q5['sales'] = df_q5['sales'].astype('float64')
df_q5

# Q6: For each store(item supplier), what are the top three products offered that are most often purchased?
    S1-> item1 -> quantity_sales

    S1-> item2 -> quantity_sales

    S1-> item3 -> quantity_sales

In [ ]:
con = PostgresConnection().getConnection()
curr = con.cursor()
select_stmt = "SELECT s.store_key as store_key , i.item_name as item_name, SUM(f.quantity) as quantity_sales "\
                "FROM star_schema.fact_table as f "\
                "JOIN star_schema.store_dim as s ON s.store_key = f.store_key " \
                "JOIN star_schema.item_dim as i ON i.item_key = f.item_key " \
                "GROUP BY CUBE(s.store_key, i.item_name) " \
                "ORDER BY s.store_key, sum(f.quantity) DESC "

curr.execute(select_stmt)
records = curr.fetchall()





In [ ]:
df_q6 = pd.DataFrame(records, columns=['store_key', 'item_name', 'quantity_sales'])
df_q6 = df_q6.dropna()
df_q6

In [ ]:
df_q6 = df_q6.groupby('store_key').head(3)
df_q6.head(20)

In [ ]:
# take top 10 data
data = df_q6[:30]
plt.figure(figsize=(20,12))
plt.title('Top 3 Items by Store')
plt.xlabel('Quantity')
plt.ylabel('Item Name')
sns.scatterplot(x='quantity_sales', y='item_name', hue='store_key', palette='tab10', s=200,data=data)
sns.set(font_scale=2.0)

In [ ]:
fig, ax = plt.subplots(figsize=(20,12))
plt.title('Top 3 Items by Store')
plt.ylabel('Quantity')
plt.xlabel('Store Key')
sns.set(font_scale=2.0)
sns.set_context("notebook")
sns.barplot(x=data['store_key'], y=data['quantity_sales'], hue=data['item_name'], palette='Paired', ax=ax)



# Q7: What products have been sold through card or mobile since X days?

    input: X = 5 days

    output: [item1, item2, item3, ...........]

In [ ]:
no_of_days = input("Enter number of days: ")

In [ ]:
con = PostgresConnection().getConnection()
curr = con.cursor()
select_stmt = "SELECT i.item_name as item_name , tim.t_date as date, tt.trans_type as trans_type " \
                "FROM star_schema.fact_table as f " \
                "JOIN star_schema.item_dim as i ON i.item_key = f.item_key " \
                "JOIN star_schema.time_dim as tim ON tim.time_key = f.time_key " \
                "JOIN star_schema.trans_dim as tt ON tt.payment_key = f.payment_key " \
                "WHERE tim.t_date > CURRENT_DATE - INTERVAL '"
select_stmt = select_stmt+ str(no_of_days) + " days' "
# select_stmt = select_stmt + str(no_of_days) + " days::interval) AND tt.trans_type = 'card' " 
print(select_stmt)
curr.execute(select_stmt)
records = curr.fetchall()
                

In [ ]:
records

In [ ]:
data = pd.DataFrame(records, columns=['item_name','date','trans_type'])
data = data.dropna()
# split the data into two parts one for card and another for mobile
data

In [ ]:
data['date'] = pd.to_datetime(data['date'])
data['trans_type'] = data['trans_type'].astype('str')
grouped = data.groupby('trans_type')
card = grouped.get_group('card')
mobile = grouped.get_group('mobile')
card


In [ ]:
mobile

In [ ]:
# import matplotlib.pyplot as plt

# Assuming you have separate DataFrames 'card' and 'mobile'

# Count the occurrences of each trans_type
card_count = card.shape[0]
mobile_count = mobile.shape[0]

# Create a list of counts
counts = [card_count, mobile_count]

# Create a list of labels
labels = ['Card', 'Mobile']

# Set the colors for the pie chart
colors = ['#ff9999', '#66b3ff']

# Create the pie chart
plt.pie(counts, labels=labels, colors=colors, autopct='%1.1f%%', startangle=90)

# Set aspect ratio to be equal so that pie is drawn as a circle
plt.axis('equal')

# Add a title
plt.title('Transaction Distribution: Card vs Mobile')

# Show the pie chart
plt.show()


# Q8: What season(quarter) is the worst for each product item? As example,

       item1-> q1

       item2-> q2

In [ ]:
con = PostgresConnection().getConnection()
curr = con.cursor()
select_stmt = "SELECT i.item_name, tim.quarter, SUM(f.total_price) as total_price " \
                "FROM star_schema.fact_table as f " \
                "JOIN star_schema.item_dim as i ON i.item_key = f.item_key " \
                "JOIN star_schema.time_dim as tim ON tim.time_key = f.time_key " \
                "GROUP BY CUBE(i.item_name, tim.quarter) " \
                "ORDER BY i.item_name, SUM(f.total_price) "

curr.execute(select_stmt)
records = curr.fetchall()


In [ ]:
# records
df_q8 = pd.DataFrame(records, columns=['item_name', 'quarter', 'total_price'])
df_q8 = df_q8.dropna()
df_q8

In [ ]:
df_q8 = df_q8.groupby('item_name').head(1)
df_q8

In [ ]:
df_q8 = df_q8[0:30]
df_q8

In [ ]:

plt.figure(figsize=(20,12))
plt.title('Worst Quarter for each item',fontsize=20)
plt.xlabel('Quantity',fontsize=20)
plt.ylabel('Item Name',fontsize=20)
sns.set(font_scale=3.0)
sns.set_context("notebook")

sns.scatterplot(x='total_price', y='item_name', hue='quarter', palette='tab10', s=200,data=df_q8)

# Q9: Break down the total sales of items geographically (division-wise).

     item1-> division1->total_sales

     item1-> division2->total_sales

In [ ]:
con = PostgresConnection().getConnection()
curr = con.cursor()
select_stmt = '''
SELECT s.division, i.item_name, sum(f.total_price)
FROM star_schema.fact_table f
JOIN star_schema.item_dim i ON i.item_key = f.item_key
JOIN star_schema.store_dim s ON s.store_key = f.store_key
GROUP BY CUBE(i.item_name, s.division)
ORDER BY i.item_name, s.division
'''
cur.execute(select_stmt)
records = cur.fetchall()



In [ ]:
records

In [ ]:
df_q8 = pd.DataFrame(records, columns=['division','item_name','sales'])
df_q8

In [ ]:
df_q8 = df_q8.dropna()
df_q8.head(30)

In [ ]:
item_div = df_q8.head(80)
fig, ax = plt.subplots(figsize = (20,12))
plt.title('Division Wise Sales',fontsize=20)
plt.xlabel('Item Name',fontsize=20)
plt.ylabel('Sales',fontsize=20)
sns.barplot(data=item_div, x='sales',y='item_name',hue='division',ax=ax,palette='Paired')

In [ ]:
# div_item = item_div
div_item = df_q8.head(80)
fig, ax = plt.subplots(figsize = (20,12))
plt.title('Division Wise Sales',fontsize=20)
plt.xlabel('Division',fontsize=20)
plt.ylabel('Sales',fontsize=20)
sns.barplot(data=div_item, y='division',x='sales',hue='item_name',ax=ax,palette='Paired')

# Q10: What are the average sales of products sales per store monthly?

      S1->M1-> avg_sales

      s1-> M2 -> avg_sales

In [ ]:
con = PostgresConnection().getConnection()
curr = con.cursor()
select_stmt = '''
    SELECT s.store_key, tim.month, avg(f.total_price)
    FROM star_schema.fact_table f
    JOIN star_schema.store_dim s ON s.store_key = f.store_key
    JOIN star_schema.time_dim tim ON tim.time_key = f.time_key
    GROUP BY CUBE(s.store_key, tim.month)
    ORDER BY s.store_key, tim.month
'''
curr.execute(select_stmt)
records = curr.fetchall()

In [ ]:
records

In [ ]:
df_q10 = pd.DataFrame(records, columns=['store_key','month','avg_sales'])
df_q10 = df_q10.dropna()
df_q10

In [ ]:
plot_data = df_q10.head(96)
plot_data

In [ ]:

fig, ax = plt.subplots(figsize = (20,12))
plt.title('Average of Store Sales Monthly')
plt.xlabel('Sales',fontsize=20)
plt.ylabel('Month',fontsize=20)
sns.set(font_scale=3.0)
sns.set_context("notebook")
sns.barplot(data=plot_data, x='month',y='avg_sales',hue='store_key',ax=ax,palette='Paired')

In [ ]:
plot_data['month'] = plot_data['month'].astype('int')
plot_data['avg_sales'] = plot_data['avg_sales'].astype('float')
plot_data['store_key'] = plot_data['store_key'].astype('str')

In [ ]:
plot_data2 = plot_data
pivot_df = plot_data2.pivot(index='store_key', columns='month', values='avg_sales')
pivot_df.T.plot(marker='o',linestyle='-', figsize=(20,12))
plt.title('Average of Store Sales Monthly')
plt.xlabel('Month',fontsize=20)
plt.ylabel('Sales',fontsize=20)
plt.show()